# 2-1

In [5]:
from transformers import AutoProcessor, AutoModelForVision2Seq, AutoModelForCausalLM, GenerationConfig
from diffusers import DiffusionPipeline
from PIL import Image
import torch


# Load Phi-4 Vision model
model_id = "microsoft/Phi-4-multimodal-instruct"
processor = AutoProcessor.from_pretrained(model_id, torch_dtype=torch.float16, trust_remote_code=True)
phi_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    device_map="cuda",
    torch_dtype="auto",
    _attn_implementation="flash_attention_2",
).to("cuda")

# Load Stable Diffusion 3 Medium pipeline
sd_model_name = "stabilityai/stable-diffusion-3-medium-diffusers"
T2Ipipe = DiffusionPipeline.from_pretrained(
    sd_model_name, torch_dtype=torch.float16
).to("cuda")

user_prompt = "<|user|>"
assistant_prompt = "<|assistant|>"
prompt_suffix = "<|end|>"



Loading pipeline components...: 100%|██████████| 9/9 [00:08<00:00,  1.08it/s]


## Prompt 1

In [ ]:
instruction = f"{user_prompt}<|image_1|>Describe the person in the picture.{prompt_suffix}{assistant_prompt}"

In [ ]:
instruction = f"{user_prompt}<|image_1|>Analyze this image and describe the person in a compact but comprehensive way. Cover multiple aspects such as facial features, hairstyle, expression, clothing style, body posture, accessories, and overall impression. Keep each aspect brief, but include as many distinct features as possible.{prompt_suffix}{assistant_prompt}"

## my photo

In [33]:
# Define content image and instruction
image_path = "myface.jpg"  # Replace with your actual image path
content_image = Image.open(image_path).convert("RGB").resize((512, 512))

# Phi-4: Generate a descriptive text prompt
inputs = processor(images=content_image, text=instruction, return_tensors="pt").to("cuda", torch.float16)
# show token count
generated_ids = phi_model.generate(
    **inputs,
    max_new_tokens=40,
    temperature=0.8,
    do_sample=True,
    top_p=0.9,
    repetition_penalty=1.1
)
description = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
# remove the prompt
print(description)
description = description[(len(instruction)-len(user_prompt)-len(prompt_suffix)-len(assistant_prompt)):]
print(f"Generated description: {description}")

# Combine description into Snoopy-style prompt
snoopy_prompt = (
    f"A Snoopy-style illustration by Charles M. Schulz of a person with following description:{description}. "
    + "in classic Peanuts comic aesthetics. "
)



print(f"Generated Snoopy-style prompt: {snoopy_prompt}")

# Stable Diffusion: Generate stylized Snoopy-style image
stylized_image = T2Ipipe(snoopy_prompt, guidance_scale=7.5).images[0]

# Save and display generated image
output_path = "snoopy_stylized_output3.png"
stylized_image = stylized_image.resize((224, 224))
stylized_image.save(output_path)
print(f"Saved Snoopy-style image: {output_path}")

Analyze this image and describe the person in a compact but comprehensive way. Cover multiple aspects such as facial features, hairstyle, expression, clothing style, body posture, accessories, and overall impression. Keep each aspect brief, but include as many distinct features as possible in bullet points.- Facial Features: Medium complexion with fair skin
- Hair: Short black hair styled neatly
- Expression: Neutral/serious
- Clothing Style: Grey t-shirt (casual)
- Accessories
Generated description: atures: Medium complexion with fair skin
- Hair: Short black hair styled neatly
- Expression: Neutral/serious
- Clothing Style: Grey t-shirt (casual)
- Accessories
Generated Snoopy-style prompt: A Snoopy-style illustration by Charles M. Schulz of a person with following description:atures: Medium complexion with fair skin
- Hair: Short black hair styled neatly
- Expression: Neutral/serious
- Clothing Style: Grey t-shirt (casual)
- Accessories. in classic Peanuts comic aesthetics. 


100%|██████████| 28/28 [00:06<00:00,  4.40it/s]


Saved Snoopy-style image: snoopy_stylized_output3.png


## CeleFaces

In [35]:
import os
from PIL import Image

# Define directories
input_folder = "content_image"
output_folder = "stylized_images1"

# Create the output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Iterate through all images in input_folder
for image_file in os.listdir(input_folder):
    if image_file.lower().endswith((".jpg", ".jpeg", ".png")):
        image_path = os.path.join(input_folder, image_file)
        
        # Load and preprocess image
        content_image = Image.open(image_path).convert("RGB").resize((512, 512))

        # Phi-4: Generate descriptive text prompt
        inputs = processor(images=content_image, text=instruction, return_tensors="pt").to("cuda", torch.float16)
        generated_ids = phi_model.generate(
            **inputs,
            max_new_tokens=35,
            temperature=0.8,
            do_sample=True,
            top_p=0.9,
            repetition_penalty=1.1
        )
        description = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        description = description[(len(instruction)-len(user_prompt)-len(prompt_suffix)-len(assistant_prompt)):]
        print(f"Generated description for {image_file}: {description}")

        # Combine description into Snoopy-style prompt
        snoopy_prompt = (
            f"A Snoopy-style illustration by Charles M. Schulz of a person with following description:{description}. "
            + "in classic Peanuts comic aesthetics. "
        )

        # Stable Diffusion: Generate stylized Snoopy-style image
        stylized_image = T2Ipipe(snoopy_prompt, guidance_scale=7.5).images[0]
        stylized_image = stylized_image.resize((224, 224))

        # Save generated image
        output_path = os.path.join(output_folder, f"{image_file}")
        stylized_image.save(output_path)
        print(f"Saved Snoopy-style image: {output_path}")

Generated description for 000079.jpg: 
- Short beard
- Slight smile on lips
- Thick eyebrows
- Light skin tone
- Black leather jacket with white stitching
- Clean-shaven


100%|██████████| 28/28 [00:06<00:00,  4.39it/s]


Saved Snoopy-style image: stylized_images1/000079.jpg
Generated description for 000020.jpg: atures': 'Short beard', 'Hair Style': 'Long hair, possibly wavy or curly', 'Expression': 'Neutral with slight furrowed brows',


100%|██████████| 28/28 [00:06<00:00,  4.39it/s]


Saved Snoopy-style image: stylized_images1/000020.jpg
Generated description for 000009.jpg: atures: Smooth skin with subtle makeup
  - Eyes: Brown eyes with dark eyeliner
  - Nose: Prominent nose
  - Mouth: Slightly smiling


...

Saved Snoopy-style image: stylized_images1/000063.jpg
Generated description for 000040.jpg: atures: 
  - Long dark hair styled upwards
  - Well-groomed eyebrows
  - Subtle makeup with neutral tone lipstick
- Expression: Neutral


100%|██████████| 28/28 [00:06<00:00,  4.31it/s]


Saved Snoopy-style image: stylized_images1/000040.jpg
Generated description for 000001.jpg: wn hair styled with volume
- Slight smile indicating happiness or contentment
- Dark eyeshadow enhancing eye brightness
- Natural makeup emphasizing cheekbones and lips
-


100%|██████████| 28/28 [00:06<00:00,  4.31it/s]


Saved Snoopy-style image: stylized_images1/000001.jpg


# 2-2

In [19]:
from PIL import Image
import torch
from diffusers import StableDiffusionImg2ImgPipeline

# Load Stable Diffusion v1-5 Img2Img pipeline
I2Ipipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to("cuda")

Loading pipeline components...: 100%|██████████| 7/7 [00:02<00:00,  2.95it/s]


## Phi-4 instruction

In [ ]:
user_prompt = "<|user|>"
assistant_prompt = "<|assistant|>"
prompt_suffix = "<|end|>"
instruction = f"{user_prompt}<|image_1|>What is shown in this image?{prompt_suffix}{assistant_prompt}"

## my photo


In [119]:


# Define content image and instruction
image_path = "myface.jpg"  # Replace with your actual image path
content_image = Image.open(image_path).convert("RGB").resize((224, 224))


# Phi-4: Generate a descriptive text prompt
inputs = processor(images=content_image, text=instruction, return_tensors="pt").to("cuda", torch.float16)
# show token count
generated_ids = phi_model.generate(
    **inputs,
    max_new_tokens=35,
    temperature=0.8,
    do_sample=True,
    top_p=0.9,
    repetition_penalty=1.1
)
description = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
# remove the prompt_suffix
description = description.replace(instruction, "").strip()
print(f"Generated description: {description}")

# Combine description into Snoopy-style prompt
snoopy_prompt = (
    f"A Snoopy-style illustration by Charles M. Schulz of {description}. "
    + "Small, curved eyes or dots, simple lines for eyebrows. "
    + "and classic Peanuts comic aesthetics. "
)


# Generate new stylized image based on your image
image = I2Ipipe(
    prompt=snoopy_prompt,
    image=content_image,
    strength=0.7,  # how much you want it changed (0.6~0.8 recommended)
    guidance_scale=10
).images[0]

# Save and view the output
output_path = "snoopy_stylized_output7.png"
image.save(output_path)
print(f"Image saved at {output_path}")


Generated description: What is shown in this image?The image features a young Asian man wearing glasses, sitting down and posing for the camera. He appears to be indoors with some chairs visible around him.

There are two other people


100%|██████████| 35/35 [00:01<00:00, 31.43it/s]
Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


Image saved at snoopy_stylized_output7.png


## prompt 2


In [ ]:
# Define content image and instruction
image_path = "myface.jpg"  # Replace with your actual image path
content_image = Image.open(image_path).convert("RGB").resize((512, 512))


# Phi-4: Generate a descriptive text prompt
inputs = processor(images=content_image, text=instruction, return_tensors="pt").to("cuda", torch.float16)
# show token count
generated_ids = phi_model.generate(
    **inputs,
    max_new_tokens=15,
    temperature=0.8,
    do_sample=True,
    top_p=0.9,
    repetition_penalty=1.1
)
description = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
# remove the prompt_suffix
description = description.replace(instruction, "").strip()
print(f"Generated description: {description}")

# Combine description into Snoopy-style prompt
snoopy_prompt = (
    f"A Snoopy-style illustration by Charles M. Schulz of a person with following description:{description}. "
    + "in classic Peanuts comic aesthetics. "
)


# Generate new stylized image based on your image
image = I2Ipipe(
    prompt=snoopy_prompt,
    image=content_image,
    strength=0.7,  # how much you want it changed (0.6~0.8 recommended)
    guidance_scale=15
).images[0]

# Save and view the output
output_path = "snoopy_stylized_output5.png"
image = image.resize((224, 224))
image.save(output_path)
print(f"Image saved at {output_path}")


In [53]:
# Define content image and instruction
image_path = "myface.jpg"  # Replace with your actual image path
content_image = Image.open(image_path).convert("RGB").resize((512, 512))


# Phi-4: Generate a descriptive text prompt
inputs = processor(images=content_image, text=instruction, return_tensors="pt").to("cuda", torch.float16)
# show token count
generated_ids = phi_model.generate(
    **inputs,
    max_new_tokens=10,
    temperature=0.8,
    do_sample=True,
    top_p=0.9,
    repetition_penalty=1.1
)
description = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
# remove the prompt_suffix
description = description[(len(instruction)-len(user_prompt)-len(prompt_suffix)-len(assistant_prompt)):]
print(f"Generated description: {description}")

# Combine description into Snoopy-style prompt
snoopy_prompt = (
    f"A Snoopy-style illustration version by Charles M. Schulz of:{description} . Keep their unique features like hairstyle, clothing style, and overall pose, but render them in the cartoonish, minimalistic style of Charles Schulz's Peanuts comic strip. The final image should look like the character belongs in the Snoopy universe—simple lines, soft colors, big expressive eyes, and a cute, dog-like cartoon form."
)


# Generate new stylized image based on your image
image = I2Ipipe(
    prompt=snoopy_prompt,
    image=content_image,
    strength=0.7,  # how much you want it changed (0.6~0.8 recommended)
    guidance_scale=7.5
).images[0]

# Save and view the output
output_path = "snoopy_stylized_output4.png"
image = image.resize((224, 224))
image.save(output_path)
print(f"Image saved at {output_path}")


The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['big expressive eyes , and a cute , dog - like cartoon form .']


Generated description: e
- Short black hair with bangs


100%|██████████| 35/35 [00:01<00:00, 26.17it/s]


Image saved at snoopy_stylized_output4.png


## Celefaces

In [120]:
import os
from PIL import Image

# Define directories
input_folder = "content_image"
output_folder = "stylized_images2"

# Create the output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Iterate through all images in input_folder
for image_file in os.listdir(input_folder):
    if image_file.lower().endswith((".jpg", ".jpeg", ".png")):
        image_path = os.path.join(input_folder, image_file)
        
        # Load and preprocess image
        content_image = Image.open(image_path).convert("RGB").resize((512, 512))

        # Phi-4: Generate descriptive text prompt
        inputs = processor(images=content_image, text=instruction, return_tensors="pt").to("cuda", torch.float16)
        generated_ids = phi_model.generate(
            **inputs,
            max_new_tokens=10,
            temperature=0.8,
            do_sample=True,
            top_p=0.9,
            repetition_penalty=1.1
        )
        description = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        description = description.replace(instruction, "").strip()
        print(f"Generated description for {image_file}: {description}")

        # Combine description into Snoopy-style prompt
        # Combine description into Snoopy-style prompt
        snoopy_prompt = (
            f"A Snoopy-style illustration version by Charles M. Schulz of:{description} . Keep their unique features like hairstyle, clothing style, and overall pose, but render them in the cartoonish, minimalistic style of Charles Schulz's Peanuts comic strip. The final image should look like the character belongs in the Snoopy universe—simple lines, soft colors, big expressive eyes, and a cute, dog-like cartoon form."
        )
        # Stable Diffusion: Generate stylized Snoopy-style image
        # Generate new stylized image based on your image
        image = I2Ipipe(
            prompt=snoopy_prompt,
            image=content_image,
            strength=0.7,  # how much you want it changed (0.6~0.8 recommended)
            guidance_scale=7.5
        ).images[0].resize((224, 224))

        # Save generated image
        output_path = os.path.join(output_folder, f"{image_file}")
        image.save(output_path)
        print(f"Saved Snoopy-style image: {output_path}")

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['the snoopy universe — simple lines , soft colors , big expressive eyes , and a cute , dog - like cartoon form .']


Generated description for 000079.jpg: What is shown in this image?The image shows a person wearing a black shirt with


100%|██████████| 35/35 [00:01<00:00, 26.13it/s]


Saved Snoopy-style image: stylized_images2/000079.jpg


The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['the snoopy universe — simple lines , soft colors , big expressive eyes , and a cute , dog - like cartoon form .']


...

Saved Snoopy-style image: stylized_images2/000040.jpg


The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['the snoopy universe — simple lines , soft colors , big expressive eyes , and a cute , dog - like cartoon form .']


Generated description for 000001.jpg: What is shown in this image?The image shows a person with long, light brown


100%|██████████| 35/35 [00:01<00:00, 25.72it/s]


Saved Snoopy-style image: stylized_images2/000001.jpg


## Prompt 2